# SHREC17 数据集处理
scenenn: ply文件 2585个
shapenet: obj文件 3678个

需要将前两者转为off文件，并区分 train val test
写了脚本`process.sh`:
```bash
#!/bin/bash
name=$(echo $1 | cut -f 1 -d '.')
ms -i $1 \
      -o "objects_off"/"$name".off \
      -s ~/caps3d/data/shrec17/test.mlx 1>/dev/null
```
这个脚本接收一个参数就可以运行了，运行放在`objects_off`文件夹下。实际运行我们希望快一点，使用`parallel`进行加速：
```bash
parallel -j 10 'bash process.sh {}' ::: `ls *.obj`
```


In [76]:
import pandas as pd
import shutil

for mode in ['train', 'test', 'validation']:
    train_csv = pd.read_csv(f"./shrec17/{mode}.csv")
    size = train_csv.loc[:,'fullId'].size

    for i in range(size):

        name = train_csv.loc[i,'fullId']
        d_ = name.split(".")[0]
        name = name.split(".")[-1]
        category = train_csv.loc[i, 'category']
        if d_ == "snn":
            source_file = f"/home/qiangzibro/caps3d/data/shrec17/scenenn/objects/objects_off/{name}.off"

        elif d_ == 'wss':
            source_file = f"/home/qiangzibro/caps3d/data/shrec17/shapenet/models/objects_off/{name}.off"
        target_file = f"/home/qiangzibro/caps3d/data/shrec17/shrec17_{mode}/{category}_{i}.off"
        if os.path.isfile(source_file):
            shutil.copy(source_file, target_file)
        else:
            print(f"unexisted file:{source_file}")

# sub_category = train_csv.loc[i, 'sub_category']